In [39]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
import sys
from sklearn.metrics import jaccard_similarity_score
import seaborn as sns
import matplotlib.pyplot as plt
import math
import os
import spacy
from spacy_langdetect import LanguageDetector
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [4]:
input = '../hotels_data.csv'

hoorah


In [40]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
sid = SentimentIntensityAnalyzer()

In [24]:
dictionary_of_lemmas = {
    "Reservations": ['booking', 'front_desk', ''], 
    "Meals": ['food', 'breakfast', 'lunch'], 
    "Location": ['city_center', 'train_station'], 
    "Services": ['cancelation', 'roomservice']
    }

df = pd.read_csv('../Data2.txt', sep=";;", header=None,error_bad_lines=False)
df.columns = ["review_id", "overall_rating", "title_of_the_review", "text_of_the_review", "username", "location_of_the_user", 
                "hotel_id", "date_stayed", "date_of_the_review"]
df['text_of_the_review'] = df.apply(lambda row: '' if pd.isna(row['text_of_the_review']) is True else row['text_of_the_review'] , axis=1)

# hotelDfassiging groundtruth to our dataset 
# the groundtruth is categorical variable ('positive', 'negative'). 
# If the rating score is 4 or 5, we say the ground truth is positive, otherwise we say it is negative.

df['groundTruth'] = df.apply(lambda row: 'positive' if float(row['overall_rating']) > 3 else 'negative' , axis=1)


hotel_ids = set()
print(df.shape)
for idx, row in df.iterrows():
    hotel_ids.add(row['hotel_id'])

print(len(hotel_ids))


/home/tomine/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
Skipping line 50647: Expected 9 fields in line 50647, saw 11. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 71068: Expected 9 fields in line 71068, saw 10. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 73612: Expected 9 fields in line 73612, saw 11. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 104136: Expected 9 fields in line 104136, saw 10. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 122095: Expected 9 fields in line 122095, saw 13. Error could

(878533, 10)
3987


In [25]:
hotel_revs = {str(hotel):[] for hotel in hotel_ids}
print(len(hotel_revs))


3987


In [28]:
for idx, row in df.iterrows():
    if pd.isna(row['hotel_id']) is False and pd.isna(row['text_of_the_review']) is False:
        hotel_revs[row['hotel_id']].append(row['text_of_the_review'])


In [ ]:
info_all_hotels = {str(hotel):[] for hotel in hotel_ids}
for hotel, reviews in hotel_revs.items():
    info_reviews = []
    for review in reviews:
#         print(review)
        doc = nlp(review)
        sentences = doc.sents
        info_sentences = []
        if doc._.language['language'] == 'en':
            for sentence in sentences:
                info_sentence = {"vaderScore":0, "lemmas": []}
                score = sid.polarity_scores(sentence.text)
                info_sentence['vaderScore'] = score['compound']
                lemmas = [tok.lemma_ for tok in sentence]
                info_sentence["lemmas"] = lemmas
                info_sentences.append(info_sentence)
#                 print(info_sentence)
        info_reviews.append(info_sentences)
    info_all_hotels[hotel].append(info_reviews)
#     print(info_all_hotels)
    



            

In [ ]:
import pickle
with open('hotel_revs.pkl', 'wb') as f:
    pickle.dump(info_all_hotels, f)